In [30]:
from pprint import pprint

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.ingestion.reader.byblock import construct_readers_byblock
from op_analytics.datapipeline.etl.ingestion.reader.request import BlockBatchRequest
from op_analytics.datapipeline.models.compute.markers import ModelsDataSpec
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

In [31]:
model_name = "teleportr"

# Prepare data raeders
data_spec = ModelsDataSpec(models=[model_name])
blockbatch_request = BlockBatchRequest.build(
    chains=["op"],
    range_spec="@20241118:+1",
    root_paths_to_read=data_spec.input_root_paths,
)
readers: list[DataReader] = construct_readers_byblock(
    blockbatch_request=blockbatch_request,
    read_from=DataLocation.GCS,
)


# Show details for the batch we are processing.
pprint(readers[0])

# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, aux_views = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0],  # use the first reader
)

2025-01-16 15:57:51 [info     ] prepared 22 input batches.     filename=byblock.py lineno=78 process=97802
DataReader(partitions=Partition(cols=[PartitionColumn(name='chain', value='op'),
                                      PartitionColumn(name='dt',
                                                      value='2024-11-18')]),
           read_from=DataLocation.GCS,
           dataset_paths={'ingestion/logs_v1': ['gs://oplabs-tools-data-sink/ingestion/logs_v1/chain=op/dt=2024-11-18/000128144000.parquet'],
                          'ingestion/transactions_v1': ['gs://oplabs-tools-data-sink/ingestion/transactions_v1/chain=op/dt=2024-11-18/000128144000.parquet']},
           inputs_ready=True,
           extra_marker_data={'max_block': 128146000,
                              'min_block': 128144000,
                              'num_blocks': 2000})
2025-01-16 15:57:51 [info     ] reading dataset='ingestion/logs_v1' using 1/1 parquet paths, first path is gs://oplabs-tools-data-sink/ingest

In [32]:
from op_analytics.datapipeline.chains.goldsky_chains import goldsky_mainnet_chains_df
from op_analytics.datapipeline.chains.across_bridge import load_across_bridge_addresses

# Fetch across bridge addesses
goldsky_df = goldsky_mainnet_chains_df()
across_bridge_addresses_df = load_across_bridge_addresses(chains_df=goldsky_df)


ctx.client.sql("CREATE TABLE IF NOT EXISTS across_bridge_metadata AS SELECT * FROM across_bridge_addresses_df")
ctx.client.sql("CREATE TABLE IF NOT EXISTS op_stack_chain_metadata AS SELECT * FROM goldsky_df")

In [33]:
ctx.client.sql("SHOW TABLES")

┌─────────────────────────┐
│          name           │
│         varchar         │
├─────────────────────────┤
│ across_bridge_metadata  │
│ op_stack_chain_metadata │
└─────────────────────────┘

In [34]:
# view = auxiliary_views["teleportr/bridging_transactions"].create_view(
#     duckdb_context=ctx,
#     template_parameters={
#         "raw_logs": input_datasets["ingestion/logs_v1"].as_subquery(),
#         "raw_transactions": input_datasets["ingestion/transactions_v1"].as_subquery(),
#         "across_bridge_metadata": "across_bridge_metadata",
#         "op_stack_chain_metadata": "op_stack_chain_metadata",
#     },
# )

In [35]:

rel = aux_views["teleportr/bridging_transactions"].to_relation(
    duckdb_context=ctx,
    template_parameters={
        "raw_logs": input_datasets["ingestion/logs_v1"].as_subquery(),
        "raw_transactions": input_datasets["ingestion/transactions_v1"].as_subquery(),
        "across_bridge_metadata": "across_bridge_metadata",
        "op_stack_chain_metadata": "op_stack_chain_metadata",
    },
)

2025-01-16 15:57:52 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=245 process=97802
2025-01-16 15:57:52 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=245 process=97802
2025-01-16 15:57:52 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=97802 template=telepotr/bridging_transactions


In [36]:
rel

┌────────────┬─────────────────┬──────────────┬───────────┬──────────────┬────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬──────────────┬─────────────────────┬─────────────────────┬─────────────────┬───────────────┬──────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬─────────────┬───────────┬───────────────────────┬───────────────────────┬───────────┬───────────────────────┐
│     dt     │ block_timestamp │ block_number │ src_chain │ src_chain_id │              contract_address              │                          transaction_hash                          │ deposit_id │            input_token_address             │            output_token_address            │ dst_chain_id │    input_amount     │    output_amount  